# Escaleras y Serpientes

<center>
    <img src="images/escalerita.png" alt="centered image" width=300/>
</center>

**Consideraciones importantes:**

* La meta del jugador es ganar la partida llegando a una de las casillas marcadas en azul.
* El jugador pierde la partida si cae en una de las casillas marcadas en rojo.
* En cada jugada, antes de lanzar el dado, el jugador decide si quiere avanzar o retroceder el número de casillas indicadas por el dado.
* En las casillas 1 y 100 la ficha rebota (si se supera el extremo, se avanza en la otra dirección la cantidad restante).

In [1]:
# Librerias que contienen la dinámica del entorno gridworld
from types import MethodType
from gridworld import GridWorld, pygame
import random
import numpy as np
import copy

pygame 2.2.0 (SDL 2.0.22, Python 3.8.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.quit
gw = None

In [11]:
gw = GridWorld(rows=10, cols=10, walls=[], pits=[(7,2),(6,3),(4,6),(1,8)], goals=[(0,0),(2,0)], live_reward=0)
serp=[(0,2),(0,5),(0,8),(1,2),(2,7),(3,3),(3,8),(4,1),(4,5),(4,8),(5,3),(5,5),(5,7)]
serpf=[(7,7),(7,3),(4,9),(8,1),(9,0),(7,3),(6,7),(8,3),(9,6),(8,9),(7,1),(9,4),(9,8)]
esca=[(9,7),(7,0),(5,9),(5,2),(4,6),(3,5),(3,1),(2,0)]
escaf=[(7,5),(1,1),(0,9),(2,3),(0,7),(1,6),(0,4),(0,0)]


In [4]:
def subpolitica (x):
    for i in gw.states:
        gw.policy[i]=x[i[0]][i[1]]
    return

In [13]:
state = gw.states
for i, s in enumerate(state):
    if s[0]%2==0 and s[1]!=0 :
        gw.policy[s] = 'W'
    elif s[0]%2==0 and s[1]==0 :
        gw.policy[s] = 'N'
    elif s[0]%2==1 and s[1]!=9 :
        gw.policy[s] = 'E'
    elif s[0]%2==1 and s[1]==9 :
        gw.policy[s] = 'N'
a=[[]]
r=[[]]

for i in range (10):
    for j in range(10):
        a[i].append('')
        r[i].append('')
    a.append([])
    r.append([])
for i in gw.states:
    a[i[0]][i[1]]=gw.policy[i]
    
for i, s in enumerate(state):
    if s[0]%2==0 and s[1]!=9 :
        gw.policy[s] = 'E'
    elif s[0]%2==0 and s[1]==9 :
        gw.policy[s] = 'S'
    elif s[0]%2==1 and s[1]!=0 :
        gw.policy[s] = 'W'
    elif s[0]%2==1 and s[1]==0 :
        gw.policy[s] = 'S'
for i in gw.states:
    r[i[0]][i[1]]=gw.policy[i]
subpolitica(a)

In [14]:
def update_values(gw, gamma):
    # Arreglos para los valores
    value = dict.fromkeys(gw.states , 0.0)
    new_values = dict.fromkeys(gw.states , 0.0)
    
    stvalues=[0,0,0,0,0,0]
    re=[0,0,0,0,0,0]
    ra=0
    sa=0
    
    for state in gw.states:
        
        stvalues=[0,0,0,0,0,0]
        re=[0,0,0,0,0,0]
        new_state=state
        
        if state in gw.goals or state in gw.pits:
            # Tomar acción de politica dada
            action = gw.policy[state]
            # Dar un paso
            new_state, reward, _, done = gw.step(state, action, random=False)
            # Actualizar valores
            # actualización para un estado terminal
            new_values[state] = reward
        elif state in serp:
            for s in serp:
                if s==state:
                    ind=serp.index(s)
                    new_values[state]=gw.state_values[serpf[ind]]
        elif state in esca:
            for s in esca:
                if s==state:
                    ind=esca.index(s)
                    new_values[state]=gw.state_values[escaf[ind]]
        else:
            for i in range(len(re)):
                action = gw.policy[new_state]
                # Dar un paso
                new_state, reward, ac, done = gw.step(new_state, action, random=False)
                
                re[i]=reward
                stvalues[i]=gw.state_values[new_state]
                # Actualizar valores
                if(new_state in gw.goals or new_state in gw.pits) and i<5:
                    
                    if new_state==(0,0):
                        subpolitica(r)
                    if new_state==(9,0):
                        subpolitica(a)
                    # actualización para un estado terminal
                    if gw.policy[new_state]=='N':
                        new_state = (new_state[0]-1, new_state[1])
                    if gw.policy[new_state]=='S':
                        new_state = (new_state[0]+1, new_state[1])
                    if gw.policy[new_state]=='W':
                        new_state = (new_state[0], new_state[1]-1)
                    if gw.policy[new_state]=='E':
                        new_state = (new_state[0], new_state[1]+1)
                    ra=gw.state_values[new_state]
                    
            subpolitica(a)
            if ra==0:
                new_values[state]=(gamma*sum(stvalues)-0.6)/6
            else:
                stvalues[5]=ra
                new_values[state]=(gamma*sum(stvalues)-0.6)/6
                ra=0
    
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values) 

In [15]:
gamma = 0.9
H = 70

In [16]:
gw.update_values = MethodType(update_values, gw)
gw.solve_dynamic_programming(gamma=gamma, horizon=H)

In [20]:
for state in gw.states:
    print(f'V{state} = {gw.state_values[state]:.3f}')

V(0, 0) = 10.000
V(0, 1) = 2.419
V(0, 2) = -0.197
V(0, 3) = 2.431
V(0, 4) = 2.431
V(0, 5) = -0.289
V(0, 6) = 2.419
V(0, 7) = 1.282
V(0, 8) = -0.058
V(0, 9) = 1.133
V(1, 0) = 0.159
V(1, 1) = 0.237
V(1, 2) = -0.225
V(1, 3) = 0.358
V(1, 4) = 0.459
V(1, 5) = 0.392
V(1, 6) = 0.508
V(1, 7) = 0.757
V(1, 8) = 0.000
V(1, 9) = 0.938
V(2, 0) = 10.000
V(2, 1) = 1.583
V(2, 2) = 1.762
V(2, 3) = 1.957
V(2, 4) = 2.197
V(2, 5) = 2.561
V(2, 6) = 2.909
V(2, 7) = -0.483
V(2, 8) = 1.535
V(2, 9) = 1.501
V(3, 0) = 0.526
V(3, 1) = 2.431
V(3, 2) = 0.253
V(3, 3) = -0.289
V(3, 4) = 0.641
V(3, 5) = 0.508
V(3, 6) = 0.628
V(3, 7) = 0.926
V(3, 8) = -0.057
V(3, 9) = 1.433
V(4, 0) = 0.511
V(4, 1) = -0.208
V(4, 2) = 0.384
V(4, 3) = 0.485
V(4, 4) = 0.519
V(4, 5) = -0.396
V(4, 6) = 0.000
V(4, 7) = 0.018
V(4, 8) = -0.343
V(4, 9) = -0.058
V(5, 0) = 0.071
V(5, 1) = 0.012
V(5, 2) = 1.957
V(5, 3) = -0.283
V(5, 4) = -0.074
V(5, 5) = -0.421
V(5, 6) = -0.052
V(5, 7) = -0.382
V(5, 8) = -0.047
V(5, 9) = 1.133
V(6, 0) = 0.089
V(6, 

In [10]:
def update_values2(gw, gamma):
    # Arreglos para los valores
    value = dict.fromkeys(gw.states , 0.0)
    new_values = dict.fromkeys(gw.states , 0.0)
    stvalues=[0,0,0,0,0,0]
    re=[0,0,0,0,0,0]
    rer=[0,0,0,0,0,0]
    stvaluesr=[0,0,0,0,0,0]
    ra=0
    sa=0
    for state in gw.states:
        subpolitica(a)
        stvalues=[0,0,0,0,0,0]
        stvaluesr=[0,0,0,0,0,0]
        rer=[0,0,0,0,0,0]
        re=[0,0,0,0,0,0]
        new_state=state
        if state in gw.goals or state in gw.pits:
            # Tomar acción de politica dada
            action = gw.policy[state]
            # Dar un paso
            new_state, reward, _, done = gw.step(state, action, random=False)
            # Actualizar valores
            # actualización para un estado terminal
            new_values[state] = reward
        elif state in serp:
            for s in serp:
                if s==state:
                    ind=serp.index(s)
                    new_values[state]=gw.state_values[serpf[ind]]
        elif state in esca:
            for s in esca:
                if s==state:
                    ind=esca.index(s)
                    new_values[state]=gw.state_values[escaf[ind]]
        else:
            for i in range(len(re)):
                action = gw.policy[new_state]
                # Dar un paso
                new_state, reward, ac, done = gw.step(new_state, action, random=False)
                re[i]=reward
                stvalues[i]=gw.state_values[new_state]
                # Actualizar valores
                if(new_state in gw.goals or new_state in gw.pits) and i<5:
                    if new_state==(0,0):
                        subpolitica(r)
                    if new_state==(9,0):
                        subpolitica(a)
                    # actualización para un estado terminal
                    
                    if gw.policy[new_state]=='N':
                        new_state = (new_state[0]-1, new_state[1])
                    if gw.policy[new_state]=='S':
                        new_state = (new_state[0]+1, new_state[1])
                    if gw.policy[new_state]=='W':
                        new_state = (new_state[0], new_state[1]-1)
                    if gw.policy[new_state]=='E':
                        new_state = (new_state[0], new_state[1]+1)
                    ra=gw.state_values[new_state]
                    
            subpolitica(r)
            if ra!=0:
                stvalues[5]=ra
                ra=0
            new_state=state
            for i in range(len(re)):
                action = gw.policy[new_state]
                # Dar un paso
                new_state, reward, ac, done = gw.step(new_state, action, random=False)
                re[i]=reward
                stvaluesr[i]=gw.state_values[new_state]
                # Actualizar valores
                if new_state==(9,0):
                        subpolitica(a)
                if(new_state in gw.goals or new_state in gw.pits) and i<5:
                    if new_state==(0,0):
                        subpolitica(r)
                    if new_state==(9,0):
                        subpolitica(a)
                    # actualización para un estado terminal
                    
                    if gw.policy[new_state]=='N':
                        new_state = (new_state[0]-1, new_state[1])
                    if gw.policy[new_state]=='S':
                        new_state = (new_state[0]+1, new_state[1])
                    if gw.policy[new_state]=='W':
                        new_state = (new_state[0], new_state[1]-1)
                    if gw.policy[new_state]=='E':
                        new_state = (new_state[0], new_state[1]+1)
                    ra=gw.state_values[new_state]
            
            if ra==0:
                av=sum(stvalues)
                retro=sum(stvaluesr)
                total=av+retro
                new_values[state]=(gamma*retro-1.2)/12
            else:
                stvaluesr[5]=ra
                av=sum(stvalues)
                retro=sum(stvaluesr)
                total=av+retro
                new_values[state]=(gamma*retro-1.2)/12
                ra=0
            print(total)
    
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values) 

In [37]:
gw.update_values = MethodType(update_values2, gw)
gw.solve_dynamic_programming(gamma=gamma, horizon=H)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
9.3
9.3
9.3
9.299999999999999
-0.7999999999999999
-0.8999999999999999
9.000000000000002
9.000000000000002
9.100000000000001
9.1
9.2
-0.8999999999999999
-0.8999999999999999
-0.8999999999999999
9.100000000000001
9.100000000000001
9.100000000000001
9.1
9.1
9.1
-0.8999999999999999
-0.8999999999999999
-0.7
-0.7
-0.7
-0.7
-0.7
-0.7999999999999999
-0.6
-0.6
-0.7
-0.6
-0.7
-0.7
-0.7999999999999999
-0.7
-0.7
-0.6
-0.5
-0.7999999999999999
-0.8999999999999999
-0.8999999999999999
-1.0999999999999999
-1.0999999999999999
-1.0999999999999999
-1.0999999999999999
-1.0999999999999999
-1.0999999999999999
-0.9999999999999999
-1.0
-1.0
-1.0
-1.0
-1.0999999999999999
-1.0999999999999999
-1.2
-1.0
-1.0
-1.0
-1.0
-1

8.673375600134888
8.771101659138184
8.98711415437439
9.369235312990723
-0.49663211695434545
1.2674681100976568
12.05047120316101
12.283956587615965
12.662394451814572
12.820428180919189
12.766211333225097
3.3721819094995116
2.721488327249146
2.0623618622485353
11.727149256723022
10.7422386956134
9.826927494465941
8.980603374134521
8.998839826228028
8.213951752836303
-1.6245512080407714
-1.6152389418621826
-1.5487795662652588
-1.525354423527832
-1.4934076675964356
-1.5015018016351318
-1.4982080951733399
-1.5455322381170653
-1.4170879208966065
-1.735216916657715
-1.7960470982385255
-1.590241129401245
-1.6388458121643066
-1.648812978796997
-1.8071869703057861
-1.812882612532959
-1.758207111930542
-1.7652117944384766
-1.6205588840673828
-1.8008932251959229
-1.7806718319738768
-1.766042587438965
-1.7627429614868164
-1.7409884600677492
-1.7173649011663819
-1.6428396622753905
-1.5744564869909667
-1.5121550634039305
-0.758471753817749
-0.7052536602606201
-0.6836276418395996
-0.6662136122509765

8.670764419773105
8.768320588446581
8.984409996150838
9.367095050299593
-0.49871953569029515
1.2662183511070384
12.052522868148094
12.285749043326074
12.663753542577197
12.821488657803204
12.766712022745027
3.3725638917011076
2.7215671790863585
2.0613951063148015
11.728511764375261
10.743649889539517
9.828302957657526
8.981804899294389
8.999507790486712
8.214301395025908
-1.6249143929165735
-1.6157996675560118
-1.5505761523085255
-1.5265438962793176
-1.4942734984732482
-1.5031860768814138
-1.4996321813121039
-1.5464300077398505
-1.4187136062053645
-1.7369748546198491
-1.7977765278775335
-1.5924508017857133
-1.6413943523098622
-1.6520376876800902
-1.8061382073712178
-1.8124274237337241
-1.7587129195975681
-1.7665309467678765
-1.6230320930442417
-1.7999889700757912
-1.7791978064180771
-1.7649917567648017
-1.7625538705604935
-1.7412311130716653
-1.7179934590108723
-1.6437733684538514
-1.5755783693807295
-1.513337852978986
-0.7579086353446289
-0.7048846451749287
-0.6834699676548095
-0.6663

8.670750642808043
8.768306288040591
8.984395975397463
9.367082094753473
-0.4987333649413701
1.2662051649557102
12.052513838348897
12.285739782409635
12.663745231589415
12.821480316631884
12.766701809359715
3.372552814790908
2.7215557373175963
2.061382377628803
11.728501116631163
10.74363893619336
9.828291605802567
8.981792990852453
8.999496396198415
8.214289160118467
-1.6249297556195348
-1.6158153996170552
-1.5505949171264117
-1.5265612033153868
-1.494290677456446
-1.5032041088880757
-1.4996497992198152
-1.5464465166447514
-1.4187310492216674
-1.73699196835435
-1.7977933540902336
-1.5924682808219544
-1.6414123779048595
-1.6520561055779623
-1.8061461624200086
-1.8124373902906783
-1.7587267621454674
-1.7665477825017002
-1.6230510384160413
-1.7999963226473281
-1.7792031325287518
-1.7649972000925125
-1.762557422085412
-1.7412353569271115
-1.7179983829577865
-1.6437790627437208
-1.5755847292131142
-1.513344768073945
-0.7579176455662772
-0.704893426058597
-0.6834786662637967
-0.6663269964700

8.670750453458503
8.768306095006515
8.98439578765591
9.367081921433893
-0.498733552348786
1.2662049810991431
12.052513664194805
12.285739609862507
12.663745083035565
12.8214801743615
12.766701646952724
3.372552641374594
2.72155556465482
2.0613821974862008
11.728500935574038
10.743638749239237
9.828291413202574
8.981792792451504
8.999496206635863
8.214288962063575
-1.624929986845338
-1.6158156342878036
-1.5505951684099892
-1.5265614533069205
-1.4942909337541308
-1.5032043650316358
-1.499650054760497
-1.546446758192653
-1.4187312813729478
-1.7369921925175515
-1.7977935712645785
-1.5924684976140828
-1.6414125900715968
-1.6520563138645659
-1.8061463093090282
-1.8124375536066455
-1.758726962516327
-1.7665480118469308
-1.6230512754568556
-1.7999964597156752
-1.7792032563255178
-1.7649973191899593
-1.7625574946354605
-1.7412354337509108
-1.7179984644286153
-1.6437791508205282
-1.5755848236201833
-1.5133448684584727
-0.7579177831404497
-0.7048935615026609
-0.6834787999933181
-0.666327128002475

...

8.670750451087272
8.768306092592447
8.984395785318403
9.367081919311318
-0.4987335546379982
1.266204978895233
12.052513662050563
12.285739607766377
12.663745081254717
12.821480172666895
12.766701645023197
3.37255263930982
2.721555562604263
2.0613821953389166
11.728500933397067
10.74363874695797
9.828291410826017
8.981792789983658
8.999496204210443
8.21428895952875
-1.6249299897627623
-1.6158156372392303
-1.5505951713056376
-1.5265614563221093
-1.4942909369254882
-1.5032043681847458
-1.499650057950677
-1.5464467612107682
-1.4187312840265878
-1.7369921950562373
-1.7977935737130926
-1.5924685000755625
-1.6414125924205984
-1.652056316143395
-1.8061463111573692
-1.812437555594908
-1.7587269649037913
-1.766548014530704
-1.6230512781290756
-1.7999964614643345
-1.7792032580047916
-1.7649973208090846
-1.7625574956865808
-1.7412354348403336
-1.7179984655571745
-1.643779152010572
-1.5755848248663011
-1.5133448697556082
-0.7579177847774846
-0.7048935631318556
-0.6834788016130525
-0.666327129609373

8.670750451087143
8.768306092592317
8.984395785318277
9.367081919311206
-0.49873355463811936
1.266204978895118
12.052513662050455
12.285739607766269
12.663745081254625
12.821480172666806
12.7667016450231
3.3725526393097143
2.7215555626041583
2.0613821953388043
11.728500933396955
10.743638746957851
9.828291410825893
8.981792789983526
8.999496204210315
8.214288959528616
-1.6249299897629164
-1.6158156372393866
-1.5505951713057793
-1.5265614563222631
-1.4942909369256547
-1.503204368184911
-1.4996500579508458
-1.5464467612109278
-1.4187312840267166
-1.7369921950563567
-1.797793573713207
-1.592468500075679
-1.6414125924207073
-1.6520563161435002
-1.8061463111574658
-1.8124375555950087
-1.7587269649039117
-1.7665480145308368
-1.6230512781292035
-1.7999964614644268
-1.7792032580048847
-1.7649973208091745
-1.7625574956866419
-1.7412354348403964
-1.7179984655572391
-1.6437791520106395
-1.5755848248663713
-1.5133448697556804
-0.7579177847775679
-0.7048935631319404
-0.6834788016131379
-0.666327129

8.670750451087143
8.768306092592317
8.984395785318275
9.367081919311206
-0.49873355463812075
1.2662049788951166
12.05251366205045
12.285739607766267
12.663745081254623
12.821480172666806
12.766701645023097
3.372552639309712
2.7215555626041565
2.0613821953388034
11.728500933396955
10.743638746957851
9.828291410825893
8.981792789983526
8.999496204210313
8.214288959528615
-1.624929989762918
-1.615815637239388
-1.5505951713057806
-1.5265614563222647
-1.4942909369256565
-1.5032043681849125
-1.4996500579508476
-1.5464467612109294
-1.4187312840267179
-1.736992195056358
-1.797793573713208
-1.59246850007568
-1.6414125924207084
-1.6520563161435011
-1.806146311157467
-1.8124375555950096
-1.7587269649039126
-1.766548014530838
-1.6230512781292046
-1.7999964614644277
-1.7792032580048858
-1.7649973208091754
-1.7625574956866426
-1.7412354348403971
-1.7179984655572398
-1.6437791520106404
-1.5755848248663722
-1.5133448697556815
-0.7579177847775687
-0.7048935631319414
-0.6834788016131388
-0.6663271296094

8.670750451087143
8.768306092592317
8.984395785318275
9.367081919311206
-0.49873355463812075
1.2662049788951166
12.05251366205045
12.285739607766267
12.663745081254623
12.821480172666806
12.766701645023097
3.372552639309712
2.7215555626041565
2.0613821953388034
11.728500933396955
10.743638746957851
9.828291410825893
8.981792789983526
8.999496204210313
8.214288959528615
-1.624929989762918
-1.615815637239388
-1.5505951713057806
-1.5265614563222647
-1.4942909369256565
-1.5032043681849125
-1.4996500579508476
-1.5464467612109294
-1.4187312840267179
-1.736992195056358
-1.797793573713208
-1.59246850007568
-1.6414125924207084
-1.6520563161435011
-1.806146311157467
-1.8124375555950096
-1.7587269649039126
-1.766548014530838
-1.6230512781292046
-1.7999964614644277
-1.7792032580048858
-1.7649973208091754
-1.7625574956866426
-1.7412354348403971
-1.7179984655572398
-1.6437791520106404
-1.5755848248663722
-1.5133448697556815
-0.7579177847775687
-0.7048935631319414
-0.6834788016131388
-0.6663271296094

8.670750451087143
8.768306092592317
8.984395785318275
9.367081919311206
-0.49873355463812075
1.2662049788951166
12.05251366205045
12.285739607766267
12.663745081254623
12.821480172666806
12.766701645023097
3.372552639309712
2.7215555626041565
2.0613821953388034
11.728500933396955
10.743638746957851
9.828291410825893
8.981792789983526
8.999496204210313
8.214288959528615
-1.624929989762918
-1.615815637239388
-1.5505951713057806
-1.5265614563222647
-1.4942909369256565
-1.5032043681849125
-1.4996500579508476
-1.5464467612109294
-1.4187312840267179
-1.736992195056358
-1.797793573713208
-1.59246850007568
-1.6414125924207084
-1.6520563161435011
-1.806146311157467
-1.8124375555950096
-1.7587269649039126
-1.766548014530838
-1.6230512781292046
-1.7999964614644277
-1.7792032580048858
-1.7649973208091754
-1.7625574956866426
-1.7412354348403971
-1.7179984655572398
-1.6437791520106404
-1.5755848248663722
-1.5133448697556815
-0.7579177847775687
-0.7048935631319414
-0.6834788016131388
-0.6663271296094